In [48]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, Float, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker


In [49]:
## data config 
SQLALCHEMY_DATABASE_URL = "sqlite:///./fantasy_data.db"

engine= create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()   

## Orm defination

In [50]:
# --- ORM Model Definitions ---

class Player(Base):
    __tablename__ = "player"
    player_id = Column(Integer, primary_key=True, nullable=False)
    gsis_id = Column(String)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    position = Column(String, nullable=False)
    last_changed_date = Column(Date, nullable=False)

class League(Base):
    __tablename__ = "league"
    league_id = Column(Integer, primary_key=True, nullable=False)
    league_name = Column(String, nullable=False)
    scoring_type = Column(String, nullable=False)
    last_changed_date = Column(Date, nullable=False)

class Performance(Base):
    __tablename__ = "performance"
    performance_id = Column(Integer, primary_key=True, nullable=False)
    week_number = Column(String, nullable=False)
    fantasy_points = Column(Float, nullable=False)
    player_id = Column(Integer, ForeignKey("player.player_id"), nullable=False)
    last_changed_date = Column(Date, nullable=False)

class Team(Base):
    __tablename__ = "team"
    team_id = Column(Integer, primary_key=True, nullable=False)
    team_name = Column(String, nullable=False)
    league_id = Column(Integer, ForeignKey("league.league_id"), nullable=False)
    last_changed_date = Column(Date, nullable=False)

class TeamPlayer(Base):
    __tablename__ = "team_player"
    # Composite primary key
    team_id = Column(Integer, ForeignKey("team.team_id"), primary_key=True, nullable=False)
    player_id = Column(Integer, ForeignKey("player.player_id"), primary_key=True, nullable=False)
    last_changed_date = Column(Date, nullable=False)

print("All ORM models defined. 🧑‍💻")

All ORM models defined. 🧑‍💻


Create Tables in the Database

In [51]:
# Create all tables in the database
Base.metadata.create_all(bind=engine)

print("Tables created successfully (if they didn't already exist). 🏗️")

Tables created successfully (if they didn't already exist). 🏗️


In [52]:
# A dictionary mapping the ORM model class to its CSV file
# This helps us automate the loading process
model_csv_map = {
    League: 'data/league_data.csv',
    Player: 'data/player_data.csv',
    Team: 'data/team_data.csv',
    Performance: 'data/performance_data.csv',
    TeamPlayer: 'data/team_player_data.csv'
}

# Load each CSV into a dictionary of DataFrames
dataframes = {}
for model, file_path in model_csv_map.items():
    table_name = model.__tablename__
    try:
        # Note: We need to parse date columns correctly
        df = pd.read_csv(file_path, parse_dates=['last_changed_date'])
        dataframes[table_name] = df
        print(f"Successfully loaded {file_path}.")
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Display a sample to verify
if 'player' in dataframes:
    print("\n--- Sample of player_data ---")
    print(dataframes['player'].head())

Successfully loaded data/league_data.csv.
Successfully loaded data/player_data.csv.
Successfully loaded data/team_data.csv.
Successfully loaded data/performance_data.csv.
Successfully loaded data/team_player_data.csv.

--- Sample of player_data ---
   player_id     gsis_id first_name last_name position last_changed_date
0       1001  00-0023459      Aaron   Rodgers       QB        2024-04-18
1       1002  00-0023853       Matt    Prater        K        2024-04-18
2       1003  00-0024243   Marcedes     Lewis       TE        2024-04-18
3       1004  00-0025565       Nick      Folk        K        2024-04-18
4       1005  00-0025580      Mason    Crosby        K        2024-04-18


In [53]:
# Create a new session to interact with the database
db = SessionLocal()

try:
    # We iterate through our map to ensure we load data for each model
    for model, file_path in model_csv_map.items():
        table_name = model.__tablename__
        if table_name in dataframes:
            df = dataframes[table_name]
            
            # Check if table is already populated to avoid duplicate data
            if db.query(model).count() > 0:
                print(f"Table '{table_name}' already contains data. Skipping insertion.")
                continue

            # Convert dataframe rows to a list of dictionaries
            records_to_insert = df.to_dict(orient='records')
            
            # Use a bulk insert for efficiency
            if records_to_insert:
                db.bulk_insert_mappings(model, records_to_insert)
                print(f"Bulk inserting {len(records_to_insert)} records into '{table_name}' table.")

    # Commit the transaction to save all changes
    db.commit()
    print("\nDatabase transaction committed. ✅")

except Exception as e:
    print(f"\nAn error occurred: {e}")
    # Roll back the transaction in case of an error
    db.rollback()
    print("Transaction rolled back.")
finally:
    # Always close the session
    db.close()
    print("Session closed.")

Bulk inserting 5 records into 'league' table.
Bulk inserting 1018 records into 'player' table.
Bulk inserting 20 records into 'team' table.
Bulk inserting 17306 records into 'performance' table.
Bulk inserting 140 records into 'team_player' table.

Database transaction committed. ✅
Session closed.


In [54]:
# Open a new session for querying
db = SessionLocal()

try:
    print("\n--- Verifying Data with ORM Queries ---")
    
    # Count players
    player_count = db.query(Player).count()
    print(f"Found {player_count} players in the database.")
    
    # Count teams
    team_count = db.query(Team).count()
    print(f"Found {team_count} teams in the database.")
    
    # Get the first player and print their name
    first_player = db.query(Player).first()
    if first_player:
        print(f"First player found: {first_player.first_name} {first_player.last_name}")

finally:
    db.close()
    print("\nVerification complete. Session closed. 🎉")


--- Verifying Data with ORM Queries ---
Found 1018 players in the database.
Found 20 teams in the database.
First player found: Aaron Rodgers

Verification complete. Session closed. 🎉
